In [1]:
import io
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd

from osgeo import ogr
import cartopy.crs as ccrs
from shapely.geometry import LineString, Point

from tathu.io import pgis
from tathu.geometry.transform import ogr2shapely
from tathu.geometry.utils import fitEllipse, extractCoordinates2NumpyArray

from my_secrets import postgis_pwd


def bytea2nparray(bytea):
    '''Converts Numpy Array from Postgres to python.'''
    bdata = io.BytesIO(bytea)
    bdata.seek(0)
    return np.load(bdata)

# OGR exceptions for conversions
ogr.UseExceptions()

In [2]:
# Load systems_filtered, systems
db = pgis.Loader("localhost", "goamazon_geo", "camilacl", postgis_pwd("camilacl"), "systems_filtered")
db_full = pgis.Loader("localhost", "goamazon_geo", "camilacl", postgis_pwd("camilacl"), "systems")
names = db.loadNames()
names_full = db_full.loadNames()
print(len(names))
print(len(names_full))

5976
91613


### From systems_filtered

In [ ]:
# From systems_filtered

query = '''SELECT name, count FROM systems_filtered ORDER BY name, date_time ASC'''
names = [q[0] for q in db.query(query)]
areas = [q[1] for q in db.query(query)]
print(len(areas))

query = '''SELECT max FROM systems_filtered ORDER BY name, date_time ASC'''
zmax = [q[0] for q in db.query(query)]
print(len(zmax))

query = '''SELECT mean FROM systems_filtered ORDER BY name, date_time ASC'''
zmean = [q[0] for q in db.query(query)]
print(len(zmean))

query = '''SELECT event, nlayers FROM systems_filtered ORDER BY name, date_time ASC'''
event = [q[0] for q in db.query(query)]
nlayers = [q[1] for q in db.query(query)]
print(len(event))

query = '''SELECT date_time FROM systems_filtered ORDER BY name, date_time ASC'''
timestamp = [q[0] for q in db.query(query)]
print(len(timestamp))

query = '''SELECT gld FROM systems_filtered ORDER BY name, date_time ASC'''
gld = [q[0] for q in db.query(query)]
print(len(gld))

query = '''SELECT name, ST_AsBinary(geom) as wkb FROM systems_filtered ORDER BY name, date_time ASC'''
geoms_names = [q[0] for q in db.query(query)]
geoms = [ogr2shapely(ogr.CreateGeometryFromWkb(bytes(q[1]))) for q in db.query(query)]
print(len(geoms))

query = (
    "SELECT relations FROM systems_filtered ORDER BY name, date_time ASC"
)
relations = [q[0] for q in db.query(query)]
print(len(relations))

query = (
    "SELECT raster, nodata FROM systems_filtered ORDER BY name, date_time ASC"
)
raster = [np.int16(bytea2nparray(q[0]))/100 for q in db.query(query)]
nodata = [np.int16(q[1])/100 for q in db.query(query)]
print(len(raster))

query = '''SELECT echotop_0, echotop_20, echotop_40, echotop0_kmmin, echotop20_kmmin, echotop40_kmmin FROM systems_filtered ORDER BY name, date_time ASC'''
echo0 = [q[0] for q in db.query(query)]
print(len(echo0))
echo20 = [q[1] for q in db.query(query)]
print(len(echo20))
echo40 = [q[2] for q in db.query(query)]
print(len(echo40))
decho0 = [q[3] for q in db.query(query)]
print(len(decho0))
decho20 = [q[4] for q in db.query(query)]
print(len(decho20))
decho40 = [q[5] for q in db.query(query)]
print(len(decho40))

query = '''SELECT viwl_kgm2, vil_kgm2, vii_kgm2 FROM systems_filtered ORDER BY name, date_time ASC'''
viwl = [bytea2nparray(q[0])/10000 for q in db.query(query)]
print(len(viwl))
vil = [bytea2nparray(q[1])/10000 for q in db.query(query)]
print(len(vil))
vii = [bytea2nparray(q[2])/10000 for q in db.query(query)]
print(len(vii))

query = '''SELECT z_freq FROM systems_filtered ORDER BY name, date_time ASC'''
zfreq = [np.int64(bytea2nparray(q[0])) for q in db.query(query)]
print(len(zfreq))

query = '''SELECT nae_s_1 FROM systems_filtered ORDER BY name, date_time ASC'''
nae = [q[0] for q in db.query(query)]
print(len(nae))

query = '''SELECT gld_strmin FROM systems_filtered ORDER BY name, date_time ASC'''
dgld = [q[0] for q in db.query(query)]
print(len(dgld))


In [4]:
systems_all = gpd.GeoDataFrame(
    {
        "name": names, 
        "timestamp": timestamp, 
        "area": areas, 
        "max_reflectivity": zmax, 
        "mean_reflectivity": zmean, 
        "has_40dbz": [True if n == 1. else False for n in nlayers], 
        "classification": event, 
        "relations": relations, 
        "gld": gld, 
        "dgld": dgld, 
        "geom": geoms, 
        "reflectivity_raster": [np.where(r == nd, np.nan, r).tolist() for r, nd in zip(raster,nodata)],
        "echotop_0": echo0, 
        "echotop_20": echo20, 
        "echotop_40": echo40, 
        "dechotop_0": decho0, 
        "dechotop_20": decho20, 
        "dechotop_40": decho40, 
        "viwl": [v.tolist() for v in viwl], 
        "vil": [v.tolist() for v in vil], 
        "vii": [v.tolist() for v in vii], 
        "zfreq": [z.tolist() for z in zfreq], 
        "nae": nae},
    geometry='geom')
systems_all.timestamp = systems_all.timestamp.dt.tz_localize("UTC")
systems_all.timestamp = systems_all.timestamp.dt.tz_convert('America/Manaus')
systems_all.timestamp = [t.isoformat() for t in systems_all.timestamp]
systems_all.nae = systems_all.nae
systems_all[["echotop_0", "echotop_20", "echotop_40"]] = systems_all[["echotop_0", "echotop_20", "echotop_40"]].astype(int)
# systems_all[["dechotop_0", "dechotop_20", "dechotop_40", "dgld"]] = systems_all[["dechotop_0", "dechotop_20", "dechotop_40", "dgld"]].round(1)
systems_all = systems_all.set_crs('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

In [ ]:
systems_all.to_file('/home/camilacl/git/tathu/sipam-tracking/systems_filtered.geojson', driver='GeoJSON')

In [6]:
sysjson = systems_all.to_json(drop_id=True)

with open('/home/camilacl/git/tathu/sipam-tracking/systems_filtered.geojson', 'w', encoding='utf-8') as file:
    file.write(sysjson)

In [7]:
test = gpd.read_file('/home/camilacl/git/tathu/sipam-tracking/systems_filtered.geojson')

In [ ]:
test.columns

### From systems

In [3]:
# From systems

query = (
    "SELECT count FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
areas_full = [q[0] for q in db_full.query(query)]
print(len(areas_full))

query = (
    "SELECT max FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
zmax_full = [q[0] for q in db_full.query(query)]
print(len(zmax_full))

query = (
    "SELECT mean FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
zmean_full = [q[0] for q in db_full.query(query)]
print(len(zmean_full))

query = (
    "SELECT event, nlayers FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
event_full = [q[0] for q in db_full.query(query)]
nlayers_full = [q[1] for q in db_full.query(query)]
print(len(event_full))

query = (
    "SELECT date_time, name FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
timestamp_full = [q[0] for q in db_full.query(query)]
namest_full = [q[1] for q in db_full.query(query)]
print(len(timestamp_full))

query = (
    "SELECT relations FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
relations_full = [q[0] for q in db_full.query(query)]
print(len(relations_full))

query = (
    "SELECT raster, nodata FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
raster_full = [np.int16(bytea2nparray(q[0]))/100 for q in db_full.query(query)]
nodata_full = [np.int16(q[1])/100 for q in db_full.query(query)]
print(len(raster_full))

query = (
    "SELECT ST_AsBinary(geom) FROM systems WHERE name NOT IN"
    " ('c8a8ed48-2db2-4eb7-b5e4-0feaf6452c5e',"
    " '1a332204-12fe-4abb-bd9d-b73f5450dd03',"
    " 'c5f70bb0-5cb3-4b09-83f9-fa003b938f65',"
    " '50f14b96-efa7-4dee-aa1b-510ece86172a') ORDER BY name, date_time ASC"
)
geoms_full = [ogr2shapely(ogr.CreateGeometryFromWkb(bytes(q[0]))) for q in db.query(query)]
print(len(geoms_full))

322896
322896
322896
322896
322896
322896
322896
322896


In [5]:
gpd.read_file("zip:///home/camilacl/git/tathu/sipam-tracking/systems_filtered.zip")

,name,timestamp,area,max_reflectivity,mean_reflectivity,has_40dbz,classification,relations,gld,dgld,...,echotop_40,dechotop_0,dechotop_20,dechotop_40,viwl,vil,vii,zfreq,nae,geometry
0,001a638b-9e79-4a0d-841c-1d11dc8f0033,2014-05-11 02:36:00-04:00,328.0,41.780000,27.880829,False,SPONTANEOUS_GENERATION,[ ],0.0,NaN,...,3,NaN,NaN,NaN,"[ [ 0.033099999999999997, 0.032899999999999999...","[ [ 0.033099999999999997, 0.032899999999999999...","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 47, 332, 208, 107, 32, 5, 2, 0...",NaN,"POLYGON ((-60.00486 -2.17682, -60.00486 -2.167..."
1,001a638b-9e79-4a0d-841c-1d11dc8f0033,2014-05-11 02:48:00-04:00,324.0,42.530000,30.231930,False,CONTINUITY,[ ],0.0,0.0,...,3,0.166667,0.000000,0.000000,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 23, 301, 169, 104, 49, 24, 3, ...",-0.000017,"POLYGON ((-59.82527 -2.10518, -59.79833 -2.105..."
2,001a638b-9e79-4a0d-841c-1d11dc8f0033,2014-05-11 03:00:00-04:00,488.0,44.289997,31.493906,False,CONTINUITY,[ ],0.0,0.0,...,4,0.000000,0.000000,0.083333,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 36, 300, 268, 161, 85, 29, 18,...",0.000467,"POLYGON ((-59.81629 -2.09623, -59.78038 -2.096..."
3,001a638b-9e79-4a0d-841c-1d11dc8f0033,2014-05-11 03:12:00-04:00,613.0,43.540000,32.626194,False,CONTINUITY,[ ],0.0,0.0,...,5,-0.333333,-0.166667,0.083333,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 36, 311, 243, 194, 109, 65, 31...",0.000283,"POLYGON ((-59.84323 -2.08727, -59.80731 -2.087..."
4,001a638b-9e79-4a0d-841c-1d11dc8f0033,2014-05-11 03:24:00-04:00,775.0,44.540000,33.253803,False,CONTINUITY,[ ],0.0,0.0,...,4,0.416667,0.166667,-0.083333,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 63, 376, 308, 196, 156, 99, 37...",0.000290,"POLYGON ((-59.87017 -2.07832, -59.82527 -2.078..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40389,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31 18:36:00-04:00,235.0,51.210000,37.144478,False,CONTINUITY,[ ],0.0,0.0,...,7,0.000000,0.000000,0.083333,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 0, 17, 85, 80, 33, 29, 20, 8, ...",-0.001170,"POLYGON ((-60.82198 -1.97981, -60.79504 -1.979..."
40390,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31 18:48:00-04:00,246.0,46.570000,34.634464,False,CONTINUITY,[ ],0.0,0.0,...,7,0.000000,0.000000,0.000000,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 0, 15, 93, 68, 42, 25, 12, 6, ...",0.000062,"POLYGON ((-60.98361 -2.06040, -60.99259 -2.060..."
40391,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31 19:00:00-04:00,283.0,49.070000,34.767525,False,CONTINUITY,[ ],0.0,0.0,...,7,0.083333,0.083333,0.000000,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 0, 7, 80, 93, 69, 27, 7, 8, 0,...",0.000182,"POLYGON ((-61.01054 -2.13205, -61.03748 -2.132..."
40392,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31 19:12:00-04:00,230.0,45.310000,35.161580,False,CONTINUITY,[ ],0.0,0.0,...,6,0.000000,0.000000,-0.083333,"[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[ [ 0, 0, 0, 0, 0, 0, 50, 99, 

In [4]:
systems_full = gpd.GeoDataFrame(
    {"name":namest_full, 
     "timestamp": timestamp_full,
     "area": areas_full, 
     "max_reflectivity": zmax_full, 
     "mean_reflectivity": zmean_full, 
     "has_40dbz": [True if n == 1. else False for n in nlayers_full], 
     "classification": event_full, 
     "relations": relations_full, 
     "geom": geoms_full, 
     "reflectivity_raster": [np.where(r == nd, np.nan, r).tolist() for r, nd in zip(raster_full,nodata_full)]},
    geometry='geom')
systems_full.timestamp = systems_full.timestamp.dt.tz_localize("UTC")
systems_full.timestamp = systems_full.timestamp.dt.tz_convert('America/Manaus')
systems_full.timestamp = [t.isoformat() for t in systems_full.timestamp]
systems_full = systems_full.set_crs('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

In [5]:
systems_full

,name,timestamp,area,max_reflectivity,mean_reflectivity,has_40dbz,classification,relations,geom,reflectivity_raster
0,00010b4c-6a1c-4539-b77a-e891b735be3b,2014-06-14T00:48:00-04:00,407.0,26.840000,21.299564,False,SPONTANEOUS_GENERATION,[],"POLYGON ((-58.88244 -4.22760, -58.90040 -4.227...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
1,00013822-cbe4-4d4d-b7bb-3abd2f56aca1,2015-02-18T11:24:00-04:00,738.0,34.790000,25.707096,False,SPONTANEOUS_GENERATION,[],"POLYGON ((-59.02611 -2.26638, -59.02611 -2.257...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2,00013822-cbe4-4d4d-b7bb-3abd2f56aca1,2015-02-18T11:36:00-04:00,302.0,31.770000,24.707397,False,SPLIT,[],"POLYGON ((-58.90938 -2.24847, -58.89142 -2.248...","[[20.6, 21.23, nan, nan, nan, nan, nan, nan, n..."
3,00024502-4d30-4ab7-854d-9433746477e7,2015-12-14T22:36:00-04:00,125.0,46.549995,36.438927,False,SPONTANEOUS_GENERATION,[],"POLYGON ((-60.46280 -3.10818, -60.42689 -3.108...","[[nan, nan, nan, nan, nan, nan, nan, 21.46, 24..."
4,00026ad2-6f6a-4280-afe3-4dfce1dca0f2,2015-02-01T12:12:00-04:00,123.0,31.900000,26.058292,False,SPLIT,[a97fb800-cdf0-4226-8c86-f9bf6fd071fc],"POLYGON ((-59.27753 -4.22760, -59.25059 -4.227...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
...,...,...,...,...,...,...,...,...,...,...
322891,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31T19:12:00-04:00,230.0,45.310000,35.161580,False,CONTINUITY,[],"POLYGON ((-60.94769 -1.89921, -60.92973 -1.899...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
322892,fffe034b-1cc8-45cd-bd17-a17379357acb,2015-07-31T19:24:00-04:00,177.0,36.400000,28.367370,False,CONTINUITY,[],"POLYGON ((-61.12728 -2.04249, -61.13625 -2.042...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
322893,fffef0ff-edf1-44a9-8d5f-51f066f37a5f,2014-03-31T15:24:00-04:00,290.0,29.470000,23.976776,False,SPONTANEOUS_GENERATION,[],"POLYGON ((-60.24730 -4.03058, -60.22934 -4.030...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
322894,ffff36ef-462c-4194-8d56-baa8a3ccedc5,2014-08-20T09:12:00-04:00,133.0,30.509998,23.613450,False,SPONTANEOUS_GENERATION,[],"POLYGON ((-59.70854 -4.44252, -59.72650 -4.442...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."


In [6]:
sysjson = systems_full.to_json(drop_id=True)

with open('/home/camilacl/git/tathu/sipam-tracking/systems.geojson', 'w', encoding='utf-8') as file:
    file.write(sysjson)